In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Importing pandas, numpy, matplotlib

Reading the csv file made by scraping the data from the website

In [2]:
cvs=pd.read_csv("torontopostal.csv")

print the contents of the file

In [3]:
cvs.columns=['PostalCode','Borough','Neighborhood']
cvs.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


Converting to data frame for more evaluation

In [4]:
post=pd.DataFrame(cvs)

In [5]:
post.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


Ignoring cells with a borough that is Not assigned.

In [6]:
post=post[post.Borough != 'Not assigned']

In [7]:
pw=np.unique(post.PostalCode)
pw.size

103

Neighborhood which has more than one postal code they are seperated by comma.

In [8]:
post=post.groupby("PostalCode").agg(lambda x: ','.join(x))

In [9]:
post.shape

(103, 2)

In [10]:
pp=post.Borough

Changing the borough with the neighborhood value whose borough is 'not assigned'

In [11]:
for i in post.Neighborhood:
    if (i=='Not assigned'):
        i=pp[i]

In [12]:
post.head(10)

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


Finding shape of the data frame

In [13]:
post.shape

(103, 2)

Reading the csv file which has the latitudes and longitudes

In [14]:
latt=pd.read_csv("Geospatial_Coordinates.csv")

In [15]:
latt=pd.DataFrame(latt)
latt.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
#latt['PostalCode']=latt['Postal Code']
latt=latt.rename(columns={'Postal Code':'PostalCode'})

In [17]:
pl=np.unique(latt.PostalCode)
pl.size

103

In [18]:
latt.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

Made inner Join with the previous data frame, post so that it can make the final data set.

In [54]:
mergein=pd.merge(left =post, right=latt, left_on='PostalCode', right_on='PostalCode' )

In [55]:
mergein.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


import nominatim from geopy to obtain latitude and longitude of Toronto

In [ ]:
from geopy.geocoders import Nominatim 

Obtaining the latitude and longitude of toronto.

In [57]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


In [27]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



In [58]:
import folium

Now we will map the neighborhoods of toronto by using latitude and longitude by the help of folium library.

In [59]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, mergein in zip(mergein['Latitude'], mergein['Longitude'], mergein['Borough'], mergein['Neighborhood']):
    label = '{}, {}'.format(mergein, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork